# Using Pulp to automate creation of teams based on rank preference

In [111]:
import pulp

**Problem:** A class has 11 students which need to be placed into 4 groups with a minimum number of 2 students per group and a maximum number of 3 students per group. Each group should be assigned a unique topic out of a list of 8 possible topics. Students may rank their topic preference with 1 being the most desirable topic and 8 being the least desireable topic and these rankings will be taken into account when creating the groups.

The possible topics are: 
* arts
* travel
* science
* nature
* health
* finance
* sports
* food

**Solution:** We can set this problem up as a constrained optimization problem where we are trying to minimize the objective function. Each of the 4 teams has a 'weight' which is the sum of the weights that the students on the team proscribed to the topic assigned to that team... our goal is to minimize the sum of the weights accross all teams. Our constraints are that each team have a minimum and maximum number of team members, that each team member can only be on 1 team, and that only 4 teams/topics can be created.

## Data
Create a dictionary of student/rankings key value pairs where the key is the students name or ID and the value is a tupple containing the rankings of the items in the order given above (i.e., arts, travel, science, nature, health, finance, sports, food).

For students who don't have a preference or do not respond the average weight can be given to all items (i.e., 4.5).

In [112]:
students = {
    'student_1': (1,2,3,5,7,6,8,4),
    'student_2': (7,2,1,4,5,3,8,6),
    'student_3': (5,4,2,1,6,3,7,8),
    'student_4': (6,4,5,7,2,8,3,1),
    'student_5': (1,4,3,2,5,6,8,7),
    'student_6': (8,2,7,5,3,6,1,4),
    'student_7': (2,3,4,7,5,8,1,6),
    'student_8': (7,6,3,5,1,4,8,2),
    'student_9': (3,5,6,1,2,4,8,7),
    'student_10': (1,6,7,5,8,3,2,4),
    'student_11': (4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5), #No Preference
}

In [113]:
# Validation
# The sum of each of the student's weights should each add up to 36
for student, preference in students.items():
    print (student, sum(preference))

student_1 36
student_2 36
student_3 36
student_4 36
student_5 36
student_6 36
student_7 36
student_8 36
student_9 36
student_10 36
student_11 36.0


In [114]:
# If we want to implement the validation programatically.
def validation_1(dictionary, items):
    # Each of the tupples should add up to the sum of the range of options.
    target = sum(range(items+1))
    #If the sum of the valeus of the tupple is equal to the target, we continue. Otherwise we throw an error.
    for key, value in dictionary.items():
        if sum(value) == target:
            pass
        else: 
            print (f'Error {key}: Rankigns do not add to {target}')

validation_1(students, 8)

In [115]:
# Create a dictionary of all student rankings. Note that we convert all of the rankings 
# to a float since one of the students (student_11) has scores in float format.
student_rankings = {}
for student, preference in students.items():
    student_rankings[student] = {
        'arts': float(preference[0]),
        'travel': float(preference[1]),
        'science': float(preference[2]),
        'nature': float(preference[3]),
        'health': float(preference[4]),
        'finance': float(preference[5]),
        'sports': float(preference[6]),
        'food': float(preference[7])
    }

## Setup LP Optimization

Here we create a pulp linear programming minimization problem.

In [126]:
problem = pulp.LpProblem("Team_Topic_Assignment", pulp.LpMinimize)

## Define Variables

We next define the variables using pulp.LpVariable which contain each of the combinations of students and topics which can be assigned.

In [127]:
assignments = pulp.LpVariable.dicts("assignment",
                                    ((student, topic) 
                                     for student in student_rankings 
                                     for topic in student_rankings[student]),
                                    cat='Binary')

In [128]:
# Validation
# 11 students can be assigned to 8 different topics, so the length of the asssignments dictionary should equal 88.
len(assignments) == 11*8

True

## Objective Function

In [129]:
# Minimize the sum of rankings
problem += pulp.lpSum(assignments[student, topic] * student_rankings[student][topic] 
                      for student in student_rankings 
                      for topic in student_rankings[student])

## Constraints

In [130]:
# Constraint: Each student is in exactly one team
for student in student_rankings:
    problem += pulp.lpSum(assignments[student, topic] 
                          for topic in student_rankings[student]) == 1

In [131]:
# Constraint: Only 4 topics are assigned, each to exactly one team
topics_assigned = 4
max_students = 3

selected_topics = pulp.LpVariable.dicts("selected_topic", 
                                        (topic for topic in next(iter(student_rankings.values())).keys()), cat='Binary')
problem += pulp.lpSum(selected_topics[topic] for topic in selected_topics) == topics_assigned  # Exactly 4 topics are selected

for topic in selected_topics:
    problem += pulp.lpSum(assignments[key, topic] for key in student_rankings) <= max_students * selected_topics[topic]  # Each selected topic is assigned to exactly one team of 3 students

# Solve

In [132]:
problem.solve()

1

# Solution

In [133]:
for combination in problem.variables():
    if combination.varValue > 0 and 'assignment' in combination.name:
        print(combination.name)

assignment_('student_1',_'arts')
assignment_('student_10',_'arts')
assignment_('student_11',_'sports')
assignment_('student_2',_'science')
assignment_('student_3',_'science')
assignment_('student_4',_'health')
assignment_('student_5',_'arts')
assignment_('student_6',_'sports')
assignment_('student_7',_'sports')
assignment_('student_8',_'health')
assignment_('student_9',_'health')
